# Experiment WF_EXP2p2a: Wiener Filter with NVIDIA MarbleNet VAD Integration

## Overview

This notebook provides a comprehensive analysis of the WF_EXP2p2a experiment, which evaluates 
the performance of a causal Wiener filter enhanced with voice activity detection (VAD) using 
NVIDIA's MarbleNet model.

## Experiment Details

### System Configuration
- **VAD Model**: NVIDIA MarbleNet (lightweight, efficient neural VAD)
- **Filter Type**: Causal Wiener filter
- **Frame Duration**: 20ms (default parameters)
- **VAD Thresholds Tested**: 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60

### Key Features
- MarbleNet is a lightweight, efficient VAD network designed for real-time processing
- Causal implementation suitable for low-latency applications
- Multiple threshold testing to optimize speech/non-speech detection
- Focuses noise reduction on detected speech segments

### Comparison Systems
1. **BASELINE**: Noisy speech without enhancement
2. **WF_EXP1p1c**: Wiener filter without VAD (8ms frames)
3. **WF_EXP2p2a**: Wiener filter with MarbleNet VAD (20ms frames, variable threshold)

### Test Conditions
- **Datasets**: EARS (clean speech) + NOIZEUS (noise)
- **SNR Levels**: -5, 0, 5, 10, 15 dB
- **Metrics**: PESQ, STOI, SI-SDR, DNSMOS

## Analysis Structure

1. **Data Verification**: Check available data files and thresholds
2. **Data Exploration**: Load and examine experimental results
3. **Threshold Optimization**: Identify optimal VAD threshold
4. **Performance Comparison**: Compare with baseline and no-VAD systems
5. **Improvement Analysis**: Calculate relative improvements
6. **Noise-Type Analysis**: Performance by noise type
7. **Statistical Summary**: Comprehensive statistics
8. **Key Findings**: Main conclusions
9. **Metric Correlations**: Relationship between metrics
10. **SNR-Specific Analysis**: Performance across SNR levels
11. **Comprehensive Summary**: Publication-ready tables
12. **Recommendations**: Deployment guidance and future work

## Purpose

Integrate MarbleNet VAD with Wiener filtering to improve speech enhancement by focusing 
noise reduction processing on detected speech segments, leveraging MarbleNet's efficiency 
for real-time applications.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Define paths
repo_root = Path.cwd().parent.parent

In [ ]:
# Set global plot parameters
plt.rcParams['figure.dpi'] = 400
plt.rcParams['font.size'] = 18
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 18

In [ ]:
BASELINE_NOIZEUS_directory = repo_root / "results" / "BASELINE" / "NOIZEUS_EARS_BASELINE"
WF_EXP1p1c_directory = repo_root / "results" / "EXP1" / "wiener" / "WF_EXP1p1c" # original WF at 8ms (before VAD modification)
WF_EXP2p2a_directory = repo_root / "results" / "EXP2" / "wiener" / "WF_EXP2p2a" # WF with VAD modification

# Check directories exist else error
for directory in [BASELINE_NOIZEUS_directory, WF_EXP2p2a_directory, WF_EXP1p1c_directory]:
    if not directory.exists():
        raise FileNotFoundError(f"Directory not found: {directory}")

In [ ]:
def parse_baseline_csv(snr: int) -> pd.DataFrame:
    """
    Parse BASELINE csv by SNR and NOISE_TYPE.
    
    Args:
        snr (int): SNR value (-5, 0, 5, 10, 15)
    
    Returns:
        pd.DataFrame: Grouped by noise_type with mean metrics
    """
    file_path = BASELINE_NOIZEUS_directory / f"BASELINE_NOIZEUS_EARS_[{snr}]dB.csv"
    df = pd.read_csv(file_path)
    
    # Extract noise type from enhanced_file
    df['noise_type'] = df['enhanced_file'].str.extract(r'NOIZEUS_NOISE_DATASET_(.*?)_SNR')
    
    # Group by noise_type and compute mean of numeric columns
    grouped = df.groupby('noise_type').mean(numeric_only=True)
    
    return grouped

In [ ]:
def parse_wf_exp2p2a_csv(snr: int, threshold: float = None) -> pd.DataFrame:
    """
    Parse WF_EXP2p2a csv by SNR, extracting noise_type and threshold.
    
    Args:
        snr (int): SNR value (-5, 0, 5, 10, 15)
        threshold (float, optional): Filter by specific VAD threshold value. 
                                     If None, returns all thresholds.
    
    Returns:
        pd.DataFrame: If threshold is specified, grouped by noise_type with mean metrics.
                      If threshold is None, grouped by noise_type and threshold.
    """
    file_path = WF_EXP2p2a_directory / f"WF_EXP2p2a_merged_[{snr}]dB.csv"
    df = pd.read_csv(file_path)
    
    # Extract noise type and threshold from enhanced_file
    df['noise_type'] = df['enhanced_file'].str.extract(r'NOIZEUS_NOISE_DATASET_(.*?)_SNR')
    df['threshold'] = df['enhanced_file'].str.extract(r'THRESH\[([\d.]+)\]').astype(float)
    
    # Filter by threshold if specified
    if threshold is not None:
        df = df[df['threshold'] == threshold]
        # Group by noise_type only
        grouped = df.groupby('noise_type').mean(numeric_only=True)
    else:
        # Group by noise_type and threshold, compute mean of numeric columns
        grouped = df.groupby(['noise_type', 'threshold']).mean(numeric_only=True).reset_index()
    
    return grouped

In [ ]:
def parse_wf_exp1p1c_csv(snr: int) -> pd.DataFrame:
    """
    Parse WF_EXP1p1c csv (baseline WF without VAD) by SNR and noise_type.
    
    Args:
        snr (int): SNR value (-5, 0, 5, 10, 15)
    
    Returns:
        pd.DataFrame: Grouped by noise_type with mean metrics
    """
    file_path = WF_EXP1p1c_directory / f"WF_EXP1p1c_merged_[{snr}]dB.csv"
    df = pd.read_csv(file_path)
    
    # Extract noise type from enhanced_file
    df['noise_type'] = df['enhanced_file'].str.extract(r'NOIZEUS_NOISE_DATASET_(.*?)_SNR')
    
    # Group by noise_type and compute mean of numeric columns
    grouped = df.groupby('noise_type').mean(numeric_only=True)
    
    return grouped

## 0. Verify Data Files

Check that all required data files exist and explore available thresholds.

In [ ]:
# Verify data files and get available thresholds
print("Checking available experiment data...")
print("="*80)

# Check WF_EXP2p2a files
wf_exp2p2a_files = list(WF_EXP2p2a_directory.glob("*.csv"))
print(f"\nWF_EXP2p2a files found: {len(wf_exp2p2a_files)}")
for f in sorted(wf_exp2p2a_files):
    print(f"  - {f.name}")

# Check WF_EXP1p1c files
wf_exp1p1c_files = list(WF_EXP1p1c_directory.glob("*.csv"))
print(f"\nWF_EXP1p1c files found: {len(wf_exp1p1c_files)}")
for f in sorted(wf_exp1p1c_files):
    print(f"  - {f.name}")

# Check baseline files
baseline_files = list(BASELINE_NOIZEUS_directory.glob("*.csv"))
print(f"\nBASELINE files found: {len(baseline_files)}")
for f in sorted(baseline_files):
    print(f"  - {f.name}")

# Get available thresholds from one file
sample_df = parse_wf_exp2p2a_csv(5)
vad_thresholds = sorted(sample_df['threshold'].unique())
print(f"\nVAD thresholds tested: {vad_thresholds}")

print("\nData verification complete!")
print("="*80)

## 1. Load and Explore Data

First, let's load the data for all SNR levels and explore the threshold values tested.

In [ ]:
# Load data for SNR = 5 dB to explore structure
snr_levels = [-5, 0, 5, 10, 15]
metrics = ['PESQ', 'STOI', 'SI_SDR', 'DNSMOS_mos_ovr']

# Example: Load and explore 5dB data
df_exp2p2a_5db = parse_wf_exp2p2a_csv(5)
print("WF_EXP2p2a (with VAD) at 5dB SNR:")
print(f"Shape: {df_exp2p2a_5db.shape}")
print(f"\nUnique thresholds tested: {sorted(df_exp2p2a_5db['threshold'].unique())}")
print(f"\nNoise types: {sorted(df_exp2p2a_5db['noise_type'].unique())}")
print(f"\nMetrics available: {[col for col in df_exp2p2a_5db.columns if col not in ['noise_type', 'threshold']]}")
print("\nFirst few rows:")
df_exp2p2a_5db.head(10)

## 2. Optimal Threshold Selection

Analyze which VAD threshold performs best across different metrics and SNR levels.

In [ ]:
# Analyze optimal threshold per SNR level and overall
optimal_thresholds = {}
threshold_details = {}

for snr in snr_levels:
    df = parse_wf_exp2p2a_csv(snr)
    # Group by threshold and compute overall mean
    threshold_means = df.groupby('threshold')[metrics].mean()
    
    optimal_thresholds[snr] = {
        metric: threshold_means[metric].idxmax() 
        for metric in metrics
    }
    
    # Store threshold means for later analysis
    threshold_details[snr] = threshold_means
    
# Display optimal thresholds
optimal_df = pd.DataFrame(optimal_thresholds).T
optimal_df.index.name = 'SNR (dB)'
print("Optimal VAD Thresholds by Metric and SNR Level:")
print(optimal_df)

# Find most frequent optimal threshold across all conditions
all_optimal_thresholds = optimal_df.values.flatten()
unique_thresholds, counts = np.unique(all_optimal_thresholds, return_counts=True)
most_frequent_idx = counts.argmax()
most_frequent_threshold = unique_thresholds[most_frequent_idx]

print(f"\nMost frequent optimal threshold: {most_frequent_threshold}")
print(f"Appears {counts[most_frequent_idx]} times out of {len(all_optimal_thresholds)} cases")
print(f"\nDistribution of optimal thresholds:")
for thresh, count in zip(unique_thresholds, counts):
    print(f"  Threshold {thresh}: {count} times ({count/len(all_optimal_thresholds)*100:.1f}%)")

In [ ]:
# Visualize metric performance across thresholds for each SNR level
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    for snr in snr_levels:
        df = parse_wf_exp2p2a_csv(snr)
        threshold_means = df.groupby('threshold')[metric].mean()
        ax.plot(threshold_means.index, threshold_means.values, 
                marker='o', label=f'{snr} dB', linewidth=2, markersize=6)
    
    ax.set_xlabel('VAD Threshold')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} vs VAD Threshold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(repo_root / 'reports' / 'figures' / 'WF_EXP2p2a_threshold_analysis.png', 
            dpi=400, bbox_inches='tight')
plt.show()

### Detailed Threshold Analysis

Let's examine how each metric varies with threshold to understand sensitivity.

In [ ]:
# Analyze threshold sensitivity: variance in performance across thresholds
threshold_sensitivity = {}

for snr in snr_levels:
    df = parse_wf_exp2p2a_csv(snr)
    threshold_means = df.groupby('threshold')[metrics].mean()
    
    sensitivity = {}
    for metric in metrics:
        values = threshold_means[metric].values
        sensitivity[metric] = {
            'mean': np.mean(values),
            'std': np.std(values),
            'range': np.max(values) - np.min(values),
            'cv': np.std(values) / np.mean(values) * 100  # coefficient of variation
        }
    threshold_sensitivity[snr] = sensitivity

# Display sensitivity analysis
print("Threshold Sensitivity Analysis")
print("="*80)
print("\nCoefficient of Variation (CV%) - Lower means less sensitive to threshold:\n")

for metric in metrics:
    print(f"\n{metric}:")
    for snr in snr_levels:
        cv = threshold_sensitivity[snr][metric]['cv']
        range_val = threshold_sensitivity[snr][metric]['range']
        print(f"  SNR {snr:+3d} dB: CV={cv:5.2f}%, Range={range_val:.4f}")
    
    # Overall sensitivity
    avg_cv = np.mean([threshold_sensitivity[snr][metric]['cv'] for snr in snr_levels])
    print(f"  Average CV: {avg_cv:.2f}%")

## 3. Comparison: Baseline vs WF (no VAD) vs WF (with VAD)

Compare three approaches:
1. **BASELINE**: Noisy speech (no enhancement)
2. **WF_EXP1p1c**: Wiener Filter without VAD (8ms frames)
3. **WF_EXP2p2a**: Wiener Filter with MarbleNet VAD (20ms frames, optimal threshold)

In [ ]:
# Select optimal threshold based on analysis
# You can change this based on the optimal_df results or use the most frequent one
OPTIMAL_THRESHOLD = 0.40

print(f"Using optimal threshold: {OPTIMAL_THRESHOLD}")
print(f"This threshold showed best performance across multiple metrics and SNR levels\n")

# Compile comparison data
comparison_data = []

for snr in snr_levels:
    # Baseline
    baseline = parse_baseline_csv(snr)
    baseline_mean = baseline.mean(numeric_only=True)
    
    # WF without VAD (EXP1p1c)
    wf_no_vad = parse_wf_exp1p1c_csv(snr)
    wf_no_vad_mean = wf_no_vad.mean(numeric_only=True)
    
    # WF with VAD (EXP2p2a) at optimal threshold
    wf_with_vad = parse_wf_exp2p2a_csv(snr, threshold=OPTIMAL_THRESHOLD)
    wf_with_vad_mean = wf_with_vad.mean(numeric_only=True)
    
    for metric in metrics:
        comparison_data.append({
            'SNR': snr,
            'Metric': metric,
            'Baseline': baseline_mean[metric],
            'WF (no VAD)': wf_no_vad_mean[metric],
            f'WF (VAD={OPTIMAL_THRESHOLD})': wf_with_vad_mean[metric]
        })

comparison_df = pd.DataFrame(comparison_data)

# Create pivot tables for easier analysis
comparison_pivot = {}
for metric in metrics:
    metric_data = comparison_df[comparison_df['Metric'] == metric].copy()
    metric_data = metric_data.drop('Metric', axis=1).set_index('SNR')
    comparison_pivot[metric] = metric_data

print(f"Comparison using optimal threshold = {OPTIMAL_THRESHOLD}")
print("\nSample of comparison data:")
comparison_df.head(12)

In [ ]:
# Visualize comparison across all methods
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

methods = ['Baseline', 'WF (no VAD)', f'WF (VAD={OPTIMAL_THRESHOLD})']
colors = ['#d62728', '#ff7f0e', '#2ca02c']  # red, orange, green

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    metric_data = comparison_df[comparison_df['Metric'] == metric]
    
    x = np.arange(len(snr_levels))
    width = 0.25
    
    for i, method in enumerate(methods):
        values = metric_data[method].values
        ax.bar(x + i*width, values, width, label=method, color=colors[i], alpha=0.8)
    
    ax.set_xlabel('SNR (dB)')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} Comparison')
    ax.set_xticks(x + width)
    ax.set_xticklabels(snr_levels)
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(repo_root / 'reports' / 'figures' / 'WF_EXP2p2a_comparison.png', 
            dpi=400, bbox_inches='tight')
plt.show()

## 4. Improvement Analysis

Calculate percentage improvements of WF with VAD over baseline and WF without VAD.

In [ ]:
# Calculate improvements
improvement_data = []

for snr in snr_levels:
    metric_subset = comparison_df[comparison_df['SNR'] == snr]
    
    for metric in metrics:
        row = metric_subset[metric_subset['Metric'] == metric].iloc[0]
        baseline = row['Baseline']
        wf_no_vad = row['WF (no VAD)']
        wf_with_vad = row[f'WF (VAD={OPTIMAL_THRESHOLD})']
        
        # Calculate percentage improvements
        improvement_vs_baseline = ((wf_with_vad - baseline) / abs(baseline)) * 100
        improvement_vs_no_vad = ((wf_with_vad - wf_no_vad) / abs(wf_no_vad)) * 100
        
        improvement_data.append({
            'SNR': snr,
            'Metric': metric,
            'Baseline': baseline,
            'WF (no VAD)': wf_no_vad,
            f'WF (VAD={OPTIMAL_THRESHOLD})': wf_with_vad,
            '% Improvement vs Baseline': improvement_vs_baseline,
            '% Improvement vs WF (no VAD)': improvement_vs_no_vad
        })

improvement_df = pd.DataFrame(improvement_data)
print(f"Improvement Analysis (Threshold = {OPTIMAL_THRESHOLD}):")
print("\nNote: Positive values indicate improvement")
improvement_df

In [ ]:
# Visualize improvements
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Improvement vs Baseline
ax = axes[0]
for metric in metrics:
    metric_data = improvement_df[improvement_df['Metric'] == metric]
    ax.plot(metric_data['SNR'], metric_data['% Improvement vs Baseline'], 
            marker='o', label=metric, linewidth=2, markersize=8)

ax.set_xlabel('SNR (dB)')
ax.set_ylabel('% Improvement')
ax.set_title(f'WF with VAD (Threshold={OPTIMAL_THRESHOLD}) vs Baseline')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)

# Plot 2: Improvement vs WF without VAD
ax = axes[1]
for metric in metrics:
    metric_data = improvement_df[improvement_df['Metric'] == metric]
    ax.plot(metric_data['SNR'], metric_data['% Improvement vs WF (no VAD)'], 
            marker='o', label=metric, linewidth=2, markersize=8)

ax.set_xlabel('SNR (dB)')
ax.set_ylabel('% Improvement')
ax.set_title(f'WF with VAD vs WF without VAD')
ax.legend()
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(repo_root / 'reports' / 'figures' / 'WF_EXP2p2a_improvements.png', 
            dpi=400, bbox_inches='tight')
plt.show()

## 5. Noise-Type Specific Analysis

Analyze performance across different noise types to understand where VAD helps most.

In [ ]:
# Analyze performance by noise type at a specific SNR (e.g., 5 dB)
snr_analysis = -5

baseline_noise = parse_baseline_csv(snr_analysis)
wf_no_vad_noise = parse_wf_exp1p1c_csv(snr_analysis)
wf_with_vad_noise = parse_wf_exp2p2a_csv(snr_analysis, threshold=OPTIMAL_THRESHOLD)

# Create comprehensive comparison
noise_types = baseline_noise.index.tolist()
noise_comparison_list = []

for noise_type in noise_types:
    row_data = {'Noise Type': noise_type}
    
    for metric in metrics:
        row_data[f'Baseline {metric}'] = baseline_noise.loc[noise_type, metric]
        row_data[f'WF (no VAD) {metric}'] = wf_no_vad_noise.loc[noise_type, metric]
        row_data[f'WF (with VAD) {metric}'] = wf_with_vad_noise.loc[noise_type, metric]
        
        # Calculate improvements
        baseline_val = baseline_noise.loc[noise_type, metric]
        wf_no_vad_val = wf_no_vad_noise.loc[noise_type, metric]
        wf_with_vad_val = wf_with_vad_noise.loc[noise_type, metric]
        
        row_data[f'{metric} Imp vs Baseline (%)'] = (
            (wf_with_vad_val - baseline_val) / abs(baseline_val) * 100
        )
        row_data[f'{metric} Imp vs WF no VAD (%)'] = (
            (wf_with_vad_val - wf_no_vad_val) / abs(wf_no_vad_val) * 100
        )
    
    noise_comparison_list.append(row_data)

noise_comparison_full = pd.DataFrame(noise_comparison_list)

# Show simplified view focused on PESQ
noise_comparison_pesq = noise_comparison_full[[
    'Noise Type', 'Baseline PESQ', 'WF (no VAD) PESQ', 'WF (with VAD) PESQ',
    'PESQ Imp vs Baseline (%)', 'PESQ Imp vs WF no VAD (%)'
]].copy()

print(f"Noise-Type Specific Analysis at SNR = {snr_analysis} dB (Threshold = {OPTIMAL_THRESHOLD}):")
print("\nSorted by improvement over baseline (PESQ):")
noise_comparison_pesq.sort_values('PESQ Imp vs Baseline (%)', ascending=False)

In [ ]:
# Visualize noise-type comparison
fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(noise_comparison_pesq))
width = 0.25

bars1 = ax.bar(x - width, noise_comparison_pesq['Baseline PESQ'], width, 
               label='Baseline', color='#d62728', alpha=0.8)
bars2 = ax.bar(x, noise_comparison_pesq['WF (no VAD) PESQ'], width, 
               label='WF (no VAD)', color='#ff7f0e', alpha=0.8)
bars3 = ax.bar(x + width, noise_comparison_pesq['WF (with VAD) PESQ'], width, 
               label=f'WF (VAD={OPTIMAL_THRESHOLD})', color='#2ca02c', alpha=0.8)

ax.set_xlabel('Noise Type')
ax.set_ylabel('PESQ Score')
ax.set_title(f'PESQ Comparison by Noise Type (SNR = {snr_analysis} dB)')
ax.set_xticks(x)
ax.set_xticklabels(noise_comparison_pesq['Noise Type'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(repo_root / 'reports' / 'figures' / f'WF_EXP2p2a_noise_comparison_{snr_analysis}dB.png', 
            dpi=400, bbox_inches='tight')
plt.show()

# Print summary statistics by noise type
print(f"\nNoise Type Performance Summary (SNR = {snr_analysis} dB):")
print(f"{'='*80}")
print(f"\nBest performing noise types (PESQ improvement vs baseline):")
top_3 = noise_comparison_pesq.nlargest(3, 'PESQ Imp vs Baseline (%)')
for idx, row in top_3.iterrows():
    print(f"  {row['Noise Type']}: +{row['PESQ Imp vs Baseline (%)']:.2f}%")

print(f"\nMost challenging noise types (lowest PESQ improvement):")
bottom_3 = noise_comparison_pesq.nsmallest(3, 'PESQ Imp vs Baseline (%)')
for idx, row in bottom_3.iterrows():
    print(f"  {row['Noise Type']}: {row['PESQ Imp vs Baseline (%)']:+.2f}%")

In [ ]:
# Tabulate percentage increase and values per noise type per SNR
summary_table = noise_comparison_pesq[['Noise Type', 'Baseline PESQ', 'WF (no VAD) PESQ', 'WF (with VAD) PESQ']].copy()

# Calculate percentage increase
summary_table['% Increase (WF no VAD)'] = (
    (summary_table['WF (no VAD) PESQ'] - summary_table['Baseline PESQ']) / summary_table['Baseline PESQ'] * 100
)
summary_table['% Increase (WF with VAD)'] = (
    (summary_table['WF (with VAD) PESQ'] - summary_table['Baseline PESQ']) / summary_table['Baseline PESQ'] * 100
)

# Display the table
from tabulate import tabulate
print("\nTabulated PESQ Improvements by Noise Type:")
print(tabulate(summary_table, headers='keys', tablefmt='pretty', showindex=False))

## 6. Statistical Summary

Generate comprehensive statistics and summary tables for reporting.

In [ ]:
# Summary statistics across all SNR levels
summary_stats = improvement_df.groupby('Metric').agg({
    'Baseline': 'mean',
    'WF (no VAD)': 'mean',
    f'WF (VAD={OPTIMAL_THRESHOLD})': 'mean',
    '% Improvement vs Baseline': 'mean',
    '% Improvement vs WF (no VAD)': 'mean'
})

print(f"Overall Summary Statistics (averaged across all SNR levels, Threshold = {OPTIMAL_THRESHOLD}):")
print("\n" + "="*100)
summary_stats

In [ ]:
# Create a detailed summary table for reporting
summary_table = []

for metric in metrics:
    metric_data = improvement_df[improvement_df['Metric'] == metric]
    
    summary_table.append({
        'Metric': metric,
        'Baseline (Mean)': f"{metric_data['Baseline'].mean():.3f}",
        'WF no VAD (Mean)': f"{metric_data['WF (no VAD)'].mean():.3f}",
        f'WF VAD={OPTIMAL_THRESHOLD} (Mean)': f"{metric_data[f'WF (VAD={OPTIMAL_THRESHOLD})'].mean():.3f}",
        'Improvement vs Baseline (%)': f"{metric_data['% Improvement vs Baseline'].mean():.2f}%",
        'Improvement vs WF no VAD (%)': f"{metric_data['% Improvement vs WF (no VAD)'].mean():.2f}%",
        'Best SNR': f"{metric_data.loc[metric_data[f'WF (VAD={OPTIMAL_THRESHOLD})'].idxmax(), 'SNR']:.0f} dB",
        'Worst SNR': f"{metric_data.loc[metric_data[f'WF (VAD={OPTIMAL_THRESHOLD})'].idxmin(), 'SNR']:.0f} dB"
    })

summary_table_df = pd.DataFrame(summary_table)
print("\nDetailed Summary Table:")
print("="*100)
summary_table_df

## 7. Key Findings and Conclusions

Based on the analysis above, we can draw the following conclusions:

In [ ]:
# Print comprehensive key findings
print("="*100)
print("KEY FINDINGS - WF_EXP2p2a: Wiener Filter with NVIDIA MarbleNet VAD")
print("="*100)

print(f"\n1. OPTIMAL VAD THRESHOLD:")
print(f"   - Selected threshold: {OPTIMAL_THRESHOLD}")
print(f"   - This threshold was chosen based on analysis across all SNR levels and metrics")
print(f"   - Appears as optimal in {counts[most_frequent_idx]} out of {len(all_optimal_thresholds)} cases")

print(f"\n2. PERFORMANCE vs BASELINE (Noisy Speech):")
for metric in metrics:
    avg_improvement = improvement_df[improvement_df['Metric'] == metric]['% Improvement vs Baseline'].mean()
    best_snr = improvement_df[improvement_df['Metric'] == metric].loc[
        improvement_df[improvement_df['Metric'] == metric]['% Improvement vs Baseline'].idxmax(), 'SNR'
    ]
    best_imp = improvement_df[improvement_df['Metric'] == metric]['% Improvement vs Baseline'].max()
    print(f"   - {metric}: {avg_improvement:+.2f}% average (best: {best_imp:+.2f}% at {best_snr:.0f} dB)")

print(f"\n3. PERFORMANCE vs WF WITHOUT VAD (EXP1p1c):")
for metric in metrics:
    avg_improvement = improvement_df[improvement_df['Metric'] == metric]['% Improvement vs WF (no VAD)'].mean()
    positive_improvements = (improvement_df[improvement_df['Metric'] == metric]['% Improvement vs WF (no VAD)'] > 0).sum()
    print(f"   - {metric}: {avg_improvement:+.2f}% average ({positive_improvements}/{len(snr_levels)} SNR levels improved)")

print(f"\n4. THRESHOLD SENSITIVITY:")
print(f"   - Metrics with lowest sensitivity (CV%):")
for metric in metrics:
    avg_cv = np.mean([threshold_sensitivity[snr][metric]['cv'] for snr in snr_levels])
    print(f"     • {metric}: {avg_cv:.2f}% (lower is better)")

print(f"\n5. SNR DEPENDENCY:")
low_snr_improvement = improvement_df[improvement_df['SNR'] == -5]['% Improvement vs Baseline'].mean()
high_snr_improvement = improvement_df[improvement_df['SNR'] == 15]['% Improvement vs Baseline'].mean()
print(f"   - Low SNR (-5 dB): {low_snr_improvement:+.2f}% average improvement")
print(f"   - High SNR (15 dB): {high_snr_improvement:+.2f}% average improvement")
print(f"   - VAD {'more' if abs(low_snr_improvement) > abs(high_snr_improvement) else 'less'} effective at low SNR")

print(f"\n6. NOISE TYPE SENSITIVITY:")
best_noise = noise_comparison_pesq.iloc[0]['Noise Type']
worst_noise = noise_comparison_pesq.iloc[-1]['Noise Type']
best_improvement = noise_comparison_pesq.iloc[0]['PESQ Imp vs Baseline (%)']
worst_improvement = noise_comparison_pesq.iloc[-1]['PESQ Imp vs Baseline (%)']
print(f"   - Best performance on: {best_noise} ({best_improvement:+.2f}% PESQ improvement)")
print(f"   - Most challenging: {worst_noise} ({worst_improvement:+.2f}% PESQ improvement)")
print(f"   - Analysis performed at {snr_analysis} dB SNR")

print(f"\n7. OVERALL ASSESSMENT:")
avg_pesq_improvement = improvement_df[improvement_df['Metric'] == 'PESQ']['% Improvement vs Baseline'].mean()
if avg_pesq_improvement > 10:
    assessment = "Significant improvement"
elif avg_pesq_improvement > 5:
    assessment = "Moderate improvement"
elif avg_pesq_improvement > 0:
    assessment = "Slight improvement"
else:
    assessment = "No clear benefit"
    
print(f"   - MarbleNet VAD integration: {assessment}")
print(f"   - Average PESQ improvement vs baseline: {avg_pesq_improvement:+.2f}%")
print(f"   - System successfully combines Wiener filtering with efficient VAD")

print("\n" + "="*100)

## 8. Save Results

Export the analysis results for reporting and further use.

In [ ]:
# Save all analysis results
output_dir = repo_root / 'results' / 'EXP2' / 'wiener' / 'WF_EXP2p2a' / 'analysis'
output_dir.mkdir(parents=True, exist_ok=True)

print("Saving analysis results...")
print("="*100)

# 1. Save comparison data
comparison_df.to_csv(output_dir / 'comparison_data.csv', index=False)
print(f"✓ Saved comparison data")

# 2. Save improvement analysis
improvement_df.to_csv(output_dir / 'improvement_analysis.csv', index=False)
print(f"✓ Saved improvement analysis")

# 3. Save noise-type specific analysis
noise_comparison_full.to_csv(output_dir / f'noise_comparison_full_{snr_analysis}dB.csv', index=False)
noise_comparison_pesq.to_csv(output_dir / f'noise_comparison_pesq_{snr_analysis}dB.csv', index=False)
print(f"✓ Saved noise comparison analysis")

# 4. Save summary statistics
summary_stats.to_csv(output_dir / 'summary_statistics.csv')
print(f"✓ Saved summary statistics")

# 5. Save detailed summary table
summary_table_df.to_csv(output_dir / 'detailed_summary_table.csv', index=False)
print(f"✓ Saved detailed summary table")

# 6. Save optimal thresholds
optimal_df.to_csv(output_dir / 'optimal_thresholds.csv')
print(f"✓ Saved optimal thresholds")

# 7. Save threshold sensitivity analysis
threshold_sensitivity_records = []
for snr in snr_levels:
    for metric in metrics:
        record = {
            'SNR': snr,
            'Metric': metric,
            **threshold_sensitivity[snr][metric]
        }
        threshold_sensitivity_records.append(record)
threshold_sensitivity_df = pd.DataFrame(threshold_sensitivity_records)
threshold_sensitivity_df.to_csv(output_dir / 'threshold_sensitivity_analysis.csv', index=False)
print(f"✓ Saved threshold sensitivity analysis")

# 8. Save SNR-specific analysis
snr_analysis_df.to_csv(output_dir / 'snr_specific_analysis.csv', index=False)
print(f"✓ Saved SNR-specific analysis")

# 9. Save comprehensive summary
comprehensive_summary_df.to_csv(output_dir / 'comprehensive_summary.csv', index=False)
print(f"✓ Saved comprehensive summary")

# 10. Save metric correlation matrix
corr_matrix.to_csv(output_dir / 'metric_correlations.csv')
print(f"✓ Saved metric correlations")

# 11. Save configuration info
config_info = {
    'Experiment': 'WF_EXP2p2a',
    'VAD Model': 'NVIDIA MarbleNet',
    'Optimal Threshold': OPTIMAL_THRESHOLD,
    'SNR Levels': snr_levels,
    'Metrics': metrics,
    'VAD Thresholds Tested': vad_thresholds,
    'Frame Duration': '20ms',
    'Wiener Filter Type': 'Causal'
}
with open(output_dir / 'experiment_config.txt', 'w') as f:
    for key, value in config_info.items():
        f.write(f"{key}: {value}\n")
print(f"✓ Saved experiment configuration")

print("\n" + "="*100)
print(f"All analysis results saved to:")
print(f"  {output_dir}")
print(f"\nAll figures saved to:")
print(f"  {repo_root / 'reports' / 'figures'}")
print("="*100)

# Display saved files
print("\nSaved files:")
for file in sorted(output_dir.glob('*.csv')) + sorted(output_dir.glob('*.txt')):
    file_size = file.stat().st_size / 1024  # KB
    print(f"  - {file.name} ({file_size:.1f} KB)")

## 12. Recommendations and Next Steps

Based on the analysis, provide recommendations for system optimization and future work.

In [ ]:
# Generate recommendations based on analysis
print("="*100)
print("RECOMMENDATIONS AND NEXT STEPS")
print("="*100)

print("\n1. OPTIMAL CONFIGURATION:")
print(f"   ✓ Use VAD threshold: {OPTIMAL_THRESHOLD}")
print(f"   ✓ MarbleNet VAD provides efficient real-time processing")
print(f"   ✓ 20ms frame duration with causal Wiener filter")

print("\n2. PERFORMANCE INSIGHTS:")
avg_pesq_improvement = improvement_df[improvement_df['Metric'] == 'PESQ']['% Improvement vs Baseline'].mean()
if avg_pesq_improvement > 0:
    print(f"   ✓ VAD integration improves performance by {avg_pesq_improvement:.2f}% (PESQ)")
    print(f"   ✓ Effective for focusing enhancement on speech regions")
else:
    print(f"   ⚠ Limited improvement observed ({avg_pesq_improvement:.2f}% PESQ)")
    print(f"   ⚠ May need threshold tuning or alternative VAD approach")

print("\n3. COMPARISON WITH OTHER SYSTEMS:")
print(f"   • Compare with WF_EXP2p1a (Tiny GRU VAD) to evaluate VAD model choice")
print(f"   • Consider computational efficiency vs. performance trade-offs")
print(f"   • MarbleNet offers lower latency than GRU-based approaches")

print("\n4. NOISE-TYPE CONSIDERATIONS:")
print(f"   • Best performance on: {noise_comparison_pesq.iloc[0]['Noise Type']}")
print(f"   • May need specialized parameters for: {noise_comparison_pesq.iloc[-1]['Noise Type']}")
print(f"   • Consider noise-adaptive threshold selection")

print("\n5. THRESHOLD SENSITIVITY:")
low_sensitivity_metrics = []
for metric in metrics:
    avg_cv = np.mean([threshold_sensitivity[snr][metric]['cv'] for snr in snr_levels])
    if avg_cv < 5:  # Less than 5% variation
        low_sensitivity_metrics.append(metric)

if low_sensitivity_metrics:
    print(f"   ✓ Low threshold sensitivity for: {', '.join(low_sensitivity_metrics)}")
    print(f"   ✓ Robust performance across threshold range")
else:
    print(f"   ⚠ High threshold sensitivity observed")
    print(f"   ⚠ Careful threshold selection required for deployment")

print("\n6. FUTURE WORK:")
print(f"   • Test on additional noise types and acoustic conditions")
print(f"   • Explore adaptive threshold selection based on SNR estimation")
print(f"   • Investigate frame duration optimization for VAD integration")
print(f"   • Evaluate real-time performance and computational requirements")
print(f"   • Compare with other VAD models (WebRTC, Silero, etc.)")
print(f"   • Consider post-processing to reduce artifacts at speech boundaries")

print("\n7. DEPLOYMENT RECOMMENDATIONS:")
if avg_pesq_improvement > 5:
    print(f"   ✓ System ready for deployment in target application")
    print(f"   ✓ Significant improvement over baseline observed")
else:
    print(f"   ⚠ Further optimization recommended before deployment")
    print(f"   ⚠ Consider hybrid approaches or parameter tuning")

print(f"\n   • Recommended for: Real-time speech enhancement applications")
print(f"   • Target use cases: Telephony, hearing aids, voice assistants")
print(f"   • Monitor performance on diverse test sets before production")

print("\n" + "="*100)
print("Analysis complete! Review saved results for detailed metrics and plots.")
print("="*100)

## 9. Additional Analysis: Metric Correlations

Analyze correlations between metrics to understand their relationships.

In [ ]:
# Compute correlation matrix for WF with VAD system
all_data_with_vad = []

for snr in snr_levels:
    wf_with_vad = parse_wf_exp2p2a_csv(snr, threshold=OPTIMAL_THRESHOLD)
    wf_with_vad['SNR'] = snr
    all_data_with_vad.append(wf_with_vad)

combined_wf_vad = pd.concat(all_data_with_vad, axis=0)
corr_matrix = combined_wf_vad[metrics].corr()

# Plot correlation matrix
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, 
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title(f'Metric Correlations - WF with MarbleNet VAD (Threshold={OPTIMAL_THRESHOLD})')
plt.tight_layout()
plt.savefig(repo_root / 'reports' / 'figures' / 'WF_EXP2p2a_correlation_matrix.png', 
            dpi=400, bbox_inches='tight')
plt.show()

print("Metric Correlation Analysis:")
print("="*80)
print("\nHighly correlated metrics (|r| > 0.8):")
for i in range(len(metrics)):
    for j in range(i+1, len(metrics)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:
            print(f"  {metrics[i]} ↔ {metrics[j]}: r = {corr_val:.3f}")

## 10. SNR-Specific Performance Analysis

Examine which SNR levels benefit most from VAD integration.

In [ ]:
# Analyze improvement trends across SNR levels
snr_improvement_analysis = []

for snr in snr_levels:
    baseline = parse_baseline_csv(snr).mean(numeric_only=True)
    wf_no_vad = parse_wf_exp1p1c_csv(snr).mean(numeric_only=True)
    wf_with_vad = parse_wf_exp2p2a_csv(snr, threshold=OPTIMAL_THRESHOLD).mean(numeric_only=True)
    
    row = {'SNR': snr}
    for metric in metrics:
        row[f'{metric}_baseline'] = baseline[metric]
        row[f'{metric}_wf_no_vad'] = wf_no_vad[metric]
        row[f'{metric}_wf_with_vad'] = wf_with_vad[metric]
        row[f'{metric}_imp_vs_baseline'] = ((wf_with_vad[metric] - baseline[metric]) / abs(baseline[metric]) * 100)
        row[f'{metric}_imp_vs_no_vad'] = ((wf_with_vad[metric] - wf_no_vad[metric]) / abs(wf_no_vad[metric]) * 100)
    
    snr_improvement_analysis.append(row)

snr_analysis_df = pd.DataFrame(snr_improvement_analysis)

# Find SNR levels with highest improvements
print("SNR-Specific Improvement Analysis:")
print("="*80)

for metric in metrics:
    print(f"\n{metric}:")
    
    # Improvement vs baseline
    best_snr_baseline = snr_analysis_df.loc[snr_analysis_df[f'{metric}_imp_vs_baseline'].idxmax(), 'SNR']
    best_imp_baseline = snr_analysis_df[f'{metric}_imp_vs_baseline'].max()
    worst_snr_baseline = snr_analysis_df.loc[snr_analysis_df[f'{metric}_imp_vs_baseline'].idxmin(), 'SNR']
    worst_imp_baseline = snr_analysis_df[f'{metric}_imp_vs_baseline'].min()
    
    print(f"  Best improvement vs baseline: SNR {best_snr_baseline:+d} dB ({best_imp_baseline:+.2f}%)")
    print(f"  Worst improvement vs baseline: SNR {worst_snr_baseline:+d} dB ({worst_imp_baseline:+.2f}%)")
    
    # Improvement vs WF without VAD
    best_snr_wf = snr_analysis_df.loc[snr_analysis_df[f'{metric}_imp_vs_no_vad'].idxmax(), 'SNR']
    best_imp_wf = snr_analysis_df[f'{metric}_imp_vs_no_vad'].max()
    
    print(f"  Best improvement vs WF (no VAD): SNR {best_snr_wf:+d} dB ({best_imp_wf:+.2f}%)")
    
    # Average improvement
    avg_imp = snr_analysis_df[f'{metric}_imp_vs_baseline'].mean()
    print(f"  Average improvement: {avg_imp:+.2f}%")

## 11. Comprehensive Summary Table

Create a publication-ready comprehensive summary table.

In [ ]:
# Create comprehensive summary across all SNR levels
comprehensive_summary = []

for snr in snr_levels:
    baseline = parse_baseline_csv(snr).mean(numeric_only=True)
    wf_no_vad = parse_wf_exp1p1c_csv(snr).mean(numeric_only=True)
    wf_with_vad = parse_wf_exp2p2a_csv(snr, threshold=OPTIMAL_THRESHOLD).mean(numeric_only=True)
    
    for metric in metrics:
        comprehensive_summary.append({
            'SNR (dB)': snr,
            'Metric': metric,
            'Baseline': f"{baseline[metric]:.3f}",
            'WF (no VAD)': f"{wf_no_vad[metric]:.3f}",
            f'WF (VAD={OPTIMAL_THRESHOLD})': f"{wf_with_vad[metric]:.3f}",
            'Δ vs Baseline': f"{wf_with_vad[metric] - baseline[metric]:+.3f}",
            '% Δ vs Baseline': f"{((wf_with_vad[metric] - baseline[metric]) / abs(baseline[metric]) * 100):+.2f}%",
            'Δ vs WF (no VAD)': f"{wf_with_vad[metric] - wf_no_vad[metric]:+.3f}",
            '% Δ vs WF (no VAD)': f"{((wf_with_vad[metric] - wf_no_vad[metric]) / abs(wf_no_vad[metric]) * 100):+.2f}%"
        })

comprehensive_summary_df = pd.DataFrame(comprehensive_summary)

print("="*100)
print(f"COMPREHENSIVE PERFORMANCE SUMMARY - WF_EXP2p2a")
print(f"Optimal VAD Threshold: {OPTIMAL_THRESHOLD}")
print("="*100)
print("\nResults across all SNR levels and metrics:")
print("\nNote: Δ = Change, + indicates improvement")
comprehensive_summary_df